In [1]:
from codecs import utf_16_be_decode
import cv2
import dlib
import imutils
from imutils import face_utils
from scipy.spatial import distance as dist 
import serial


def eye_aspect_ratio(eye):
    two_and_six = dist.euclidean(eye[1], eye[5]) 
    three_and_five = dist.euclidean(eye[2], eye[4])
    one_and_four = dist.euclidean(eye[0], eye[3])
    ear = (two_and_six + three_and_five) / (2.0 * one_and_four)
    return ear 

ser = serial.Serial(port='COM17', baudrate =9600, bytesize=8, timeout=2,stopbits=serial.STOPBITS_ONE)

FACIAL_LANDMARK_PREDICTOR = "shape_predictor_68_face_landmarks.dat"  
MINIMUM_EAR = 0.2

faceDetector = dlib.get_frontal_face_detector()
landmarkFinder = dlib.shape_predictor(FACIAL_LANDMARK_PREDICTOR)
webcamFeed = cv2.VideoCapture(0)
    
(leftEyeStart, leftEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rightEyeStart, rightEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
EYE_CLOSED_COUNTER = 0
i=0
while True:
    (status, image) = webcamFeed.read()
    image = imutils.resize(image, width=800)
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = faceDetector(grayImage, 0)

    for face in faces:
        faceLandmarks = landmarkFinder(grayImage, face)
        faceLandmarks = face_utils.shape_to_np(faceLandmarks)

        leftEye = faceLandmarks[leftEyeStart:leftEyeEnd]
        rightEye = faceLandmarks[rightEyeStart:rightEyeEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        cv2.drawContours(image, [leftEye], -1, (0, 255, 0), 2)
        cv2.drawContours(image, [rightEye], -1, (0, 255, 0), 2)

        # alcohol = ser.read()
        # alcohol1= alcohol.decode('ISO-8859-1')
        # alcoholnumber = int(alcohol1)
        # print(alcohol)
        # # if alcoholnumber<500:
        #     print("The driver is drunk")
            
        if ear < MINIMUM_EAR:
            EYE_CLOSED_COUNTER = EYE_CLOSED_COUNTER+1

        cv2.putText(image, "Number of blinks " + str(EYE_CLOSED_COUNTER),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        if ear < MINIMUM_EAR: 
            i=i+1
            if i>9:
                cv2.putText(image, "Driver Asleep",(10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 4)
                
                ser.write("1".encode())
               
        else:
            i=0  
            ser.write("0".encode())   

           
        
                
             

    cv2.imshow("Frame", image)
    key = cv2.waitKey(1)
    
    if key & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break